- 一种 wrapper、装饰器设计模式
    - 这种 wrapper 一般是一种非常轻量，非常 utils 的适配，比如监控函数执行的时间；
    - 内部函数（wrapper/inner）是为被装饰的函数对象增加一些额外的属性（additional features）
- 装饰器
    - 函数装饰器（function decorator），装饰一个函数
    - 类装饰器（class decorator），装饰一个类
        - 类内部的成员函数也可以有（method decorator）
- 装饰器形式上是一个函数（或者一个支持 `__call__`的类），接受一个函数对象（入参），返回一个函数对象（内部函数对象）；
    - 使用上，可以通过 `@`注解来使用；

## 函数（对象）作为一等公民

### 函数作为参数

In [2]:
def compose(f, g, x):
    return f(g(x))
compose(print, len, [1, 2, 3])
compose(print, len, 'hello world!')

3
12


### 嵌套函数

In [78]:
import random
def random_power():
    # f, g, h: nested functions
    def f(x):
        return x**2
    def g(x):
        return x**3
    def h(x):
        return x**4
    funcs = [f, g, h]
    return random.choice(funcs, )
print(random_power())
print(random_power().__name__)
print(random_power()(3))

<function random_power.<locals>.h at 0x7f4776304ca0>
h
27


## 一个示例

In [79]:
import time
def worker1():
    time.sleep(1.5)
def worker2():
    time.sleep(1.2)
worker1()
worker2()

In [34]:
def tictoc(func):
    # nested function
    def wrapper():
        t1 = time.time()
        func()
        t2 = time.time()
        print(f'{func.__name__} Took {t2-t1} seconds')
    return wrapper

In [80]:
def worker1():
    time.sleep(1.5)
tictoc(worker1)

<function __main__.tictoc.<locals>.wrapper(*args, **kwargs)>

In [81]:
tictoc(worker1)()

worker1 Took 1.506929636001587 seconds


In [82]:
@tictoc
def worker1():
    time.sleep(1.5)
@tictoc
def worker2():
    time.sleep(1.2)
worker1()
worker2()

worker1 Took 1.5075702667236328 seconds
worker2 Took 1.2060577869415283 seconds


## 带参数, 返回值

In [83]:
def prime_factorizaton(n):
    factors = []
    divisor = 2
    while n > 1:
        while n % divisor == 0:
            factors.append(divisor)
            n //= divisor 
        divisor += 1
    return factors

In [84]:
prime_factorizaton(123)

[3, 41]

In [40]:
def timer(func):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        
        # 核心调用
        results = func(*args, **kwargs)
        
        t2 = time.time()
        print(f'{func.__name__} Took {t2-t1} seconds')
        return results
    return wrapper

In [44]:
def prime_factorizaton(n):
    factors = []
    divisor = 2
    while n > 1:
        while n % divisor == 0:
            factors.append(divisor)
            n //= divisor 
        divisor += 1
    return factors

In [85]:
timer(prime_factorizaton)

<function __main__.timer.<locals>.wrapper(*args, **kwargs)>

In [86]:
timer(prime_factorizaton)(n=2**29+1)

prime_factorizaton took 0.304431676864624 seconds


[3, 59, 3033169]

In [87]:
@timer
def prime_factorizaton(n):
    factors = []
    divisor = 2
    while n > 1:
        while n % divisor == 0:
            factors.append(divisor)
            n //= divisor 
        divisor += 1
    return factors

In [88]:
factors = prime_factorizaton(n=2**29+1)
factors

prime_factorizaton took 0.3085780143737793 seconds


[3, 59, 3033169]

## `functools`

- `*args, **kwargs`：兼容所有参数

    ```
    def decorator(func):
        def nested_inner_func(*args, **kwargs):
            ...
            results = func(*args, **kwargs)
            ...
            return results
        return nested_inner_func

    @decorator
    def f(x):
        ...

    decorator(f)(x)
    f(x)
    ```

In [89]:
import functools
for f in dir(functools):
    if f.startswith('_'):
        continue
    print(f)

GenericAlias
RLock
WRAPPER_ASSIGNMENTS
WRAPPER_UPDATES
cache
cached_property
cmp_to_key
get_cache_token
lru_cache
namedtuple
partial
partialmethod
recursive_repr
reduce
singledispatch
singledispatchmethod
total_ordering
update_wrapper
wraps


In [58]:
def func_args(*args, **kwargs):
    ''' view all args '''
    print(f'args={args}, kwargs={kwargs}')
func_args('a', 1, 'hello', n=100, name='zhang')

args=('a', 1, 'hello'), kwargs={'n': 100, 'name': 'zhang'}


In [90]:
func_args.__name__

'func_args'

In [60]:
func_args.__doc__

' view all args '

In [93]:
def do_nothing(f):
    def inner(*args, **kwargs):
        ''' inner function '''
        return f(*args, **kwargs)
    return inner

@do_nothing
def func_args(*args, **kwargs):
    ''' view all args '''
    print(f'args={args}, kwargs={kwargs}')

In [94]:
print(func_args.__name__)
print(func_args.__doc__)

inner
 inner function 


### `functools.wraps`

In [95]:
from functools import wraps

def do_nothing(f):
    
    @wraps(f)
    def inner(*args, **kwargs):
        ''' inner function '''
        return f(*args, **kwargs)
    return inner

@do_nothing
def func_args(*args, **kwargs):
    ''' view all args '''
    print(f'args={args}, kwargs={kwargs}')
    
print(func_args.__name__)
print(func_args.__doc__)

func_args
 view all args 


### `functools.cache`

In [100]:
from functools import cache

@cache
def fibonacci(n):
    # starts from 1, 1, 
    if n <= 2:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

import time
def timer(f):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = f(*args, **kwargs)
        t2 = time.time()
        print(f'{f.__name__} took {t2 - t1} seconds')
        return result
    return wrapper

@timer
def global_fibonacci(n):
    return fibonacci(n)

In [101]:
global_fibonacci(40)

global_fibonacci took 5.173683166503906e-05 seconds


102334155

In [106]:
# 查看 cache 信息
fibonacci.cache_info()

CacheInfo(hits=37, misses=40, maxsize=None, currsize=40)

In [70]:
def my_cache(f):
    memo = {}
    def wrapper(n):
        if n not in memo:
            memo[n] = f(n)
        else:
            print(f'hit memo[{n}]')
        return memo[n]
    return wrapper

In [98]:
@my_cache
def fibonacci(n):
    # starts from 1, 1, 
    if n <= 2:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)


def timer(f):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = f(*args, **kwargs)
        t2 = time.time()
        print(f'{f.__name__} took {t2 - t1} seconds')
        return result
    return wrapper

@timer
def global_fibonacci(n):
    return fibonacci(n)

In [99]:
global_fibonacci(40)

hit memo[2]
hit memo[3]
hit memo[4]
hit memo[5]
hit memo[6]
hit memo[7]
hit memo[8]
hit memo[9]
hit memo[10]
hit memo[11]
hit memo[12]
hit memo[13]
hit memo[14]
hit memo[15]
hit memo[16]
hit memo[17]
hit memo[18]
hit memo[19]
hit memo[20]
hit memo[21]
hit memo[22]
hit memo[23]
hit memo[24]
hit memo[25]
hit memo[26]
hit memo[27]
hit memo[28]
hit memo[29]
hit memo[30]
hit memo[31]
hit memo[32]
hit memo[33]
hit memo[34]
hit memo[35]
hit memo[36]
hit memo[37]
hit memo[38]
global_fibonacci took 0.00046133995056152344 seconds


102334155